In [3]:
import numpy as np
from tools import SparseGroupLasso
from sklearn.linear_model import *

import matplotlib as mpl

mpl.use('TkAgg')
import matplotlib.pyplot as plt
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 12}
mpl.rc('font', **font)

## Step 0. Load PDS-specific data

In [17]:
fm = np.load('../data/fm.npy')
e = np.load('../data/e.npy')
e = e.reshape(-1,1)
groups = np.load('../data/groups.npy')
ZERO_CLUSTER = 0
FIRST_POINT_CLUSTER = 1 
LAST_POINT_CLUSTER = 8 

## Step 1. Generate wraparound functions for 1) hyperparameter optimization and 2) orbits vs. RMSE, orbits vs. features

In [106]:
def sgl_hyperparameter_optimization(alpha, lambda_mixing, groups,
                                    X_train, X_test, 
                                    y_train, y_test):
    E0 = np.mean(y_train)
    E1_lasso_CV = LassoCV(fit_intercept = True).fit(
        X_train[:, FIRST_POINT_CLUSTER:LAST_POINT_CLUSTER], y_train-E0)
    E1 = E1_lasso_CV.predict(
        X_train[:, FIRST_POINT_CLUSTER:LAST_POINT_CLUSTER]).reshape(-1,1)
    E1_predict = E1_lasso_CV.predict(X_test[:, FIRST_POINT_CLUSTER:LAST_POINT_CLUSTER]).reshape(-1,1)
    
    y_train_remainder = y_train - E0 - E1
    y_test_remainder = y_test - E0 - E1_predict
    
    sgl = SparseGroupLasso(groups=groups[LAST_POINT_CLUSTER:],
                          lambd=lambda_mixing)
    sgl.set_params(alpha)
    sgl.fit(X_train[:, LAST_POINT_CLUSTER:], 
            y_train_remainder)
    
    y_train_predict = np.dot(X_train[:, LAST_POINT_CLUSTER:], sgl.coef_)
    y_test_predict = np.dot(X_test[:, LAST_POINT_CLUSTER:], sgl.coef_)
    
    rmse_train = np.sqrt(mean_squared_error(y_train_predict.ravel(), y_train_remainder.ravel()))
    rmse_test = np.sqrt(mean_squared_error(y_test_predict.ravel(), y_test_remainder.ravel()))
    no_features = len(np.where(np.abs(sgl.coef_) > CUTOFF)[0])

    return [rmse_train, rmse_test, no_features]

## Step 2. Run many trials

In [111]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

TRIALS = 10
TEST_SIZE = 0.2
LAMBDA_MIXING = 0.5

train, test, features = [], [], []
hyperparameters = np.logspace(-4,-2)
for penalty in hyperparameters: 
    penalty_train, penalty_test, penalty_features = [], [], [] 
    for _ in range(TRIALS):
        X_train, X_test, y_train, y_test = train_test_split(fm, e, test_size=TEST_SIZE)
        results = sgl_hyperparameter_optimization(penalty, LAMBDA_MIXING, groups,
                                                  X_train, X_test, y_train, y_test)
        
        [rmse_train, rmse_test, no_features] = results
        penalty_train.append(rmse_train)
        penalty_test.append(rmse_test)
        penalty_features.append(no_features)
    train.append(np.mean(penalty_train))
    test.append(np.mean(penalty_test))
    features.append(np.mean(penalty_features))

fig = plt.figure()
ax1 = fig.add_subplot(111)
ax2 = ax1.twinx()
ax1.plot(hyperparameters, train, color = 'green', marker = 'x')
ax1.plot(hyperparameters, test, color = 'red', marker = 'x')
ax2.plot(hyperparameters, features, color = 'orange', marker = 'x')
ax2.set_ylim([0, 450])
ax2.axhline(y=428, ls = '--', color = 'black')
plt.show()

/Users/jhyang/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/jhyang/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.530087174978257, tolerance: 0.00043918700318721515
  model = cd_fast.enet_coordinate_descent(
/Users/jhyang/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.39379979072057747, tolerance: 0.00038231550150674833
  model = cd_fast.enet_coordinate_descent(
/Users/jhyang/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: Con

/Users/jhyang/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/jhyang/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.37898220406925276, tolerance: 0.00041055428152284414
  model = cd_fast.enet_coordinate_descent(
/Users/jhyang/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.22497834386789428, tolerance: 0.00036575341919636097
  model = cd_fast.enet_coordinate_descent(
/Users/jhyang/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: C

/Users/jhyang/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/jhyang/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.8776511344471571, tolerance: 0.00045495556591622664
  model = cd_fast.enet_coordinate_descent(
/Users/jhyang/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.7845707618686344, tolerance: 0.00040697206343984097
  model = cd_fast.enet_coordinate_descent(
/Users/jhyang/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: Con

/Users/jhyang/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/jhyang/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.01206839937069848, tolerance: 0.0004367291938920196
  model = cd_fast.enet_coordinate_descent(
/Users/jhyang/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.09558187305262811, tolerance: 0.0002940362748653359
  model = cd_fast.enet_coordinate_descent(
/Users/jhyang/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: Con

/Users/jhyang/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/jhyang/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0033837767071291935, tolerance: 0.0004690332466992901
  model = cd_fast.enet_coordinate_descent(
/Users/jhyang/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.17095294081698942, tolerance: 0.000456989438618771
  model = cd_fast.enet_coordinate_descent(
/Users/jhyang/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: Co

...

/Users/jhyang/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/jhyang/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/jhyang/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/jhyang/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of

/Users/jhyang/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/jhyang/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/jhyang/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/jhyang/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of

/Users/jhyang/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/jhyang/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/jhyang/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/jhyang/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of

/Users/jhyang/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/jhyang/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/jhyang/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/jhyang/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of

KeyboardInterrupt: 

### Running the above results in a hyperparameter of $\approx \alpha = 0.002$ for a mixing of $\lambda = 0.5$ with train/test RMSE of $0.082/0.065$ eV/prim for 100 total  features. 

## Step 3. Save all coefficients 

In [137]:
ALPHA = 0.002
X_train, X_test, y_train, y_test = train_test_split(fm, e, 
                                                    test_size=TEST_SIZE)
        
E0 = np.mean(y_train)
E1_lasso_CV = LassoCV(fit_intercept = True).fit(
        X_train[:, FIRST_POINT_CLUSTER:LAST_POINT_CLUSTER], y_train.ravel()-E0)
E1 = E1_lasso_CV.predict(
        X_train[:, FIRST_POINT_CLUSTER:LAST_POINT_CLUSTER]).reshape(-1,1)
E1_predict = E1_lasso_CV.predict(X_test[:, FIRST_POINT_CLUSTER:LAST_POINT_CLUSTER]).reshape(-1,1)
    
y_train_remainder = y_train - E0 - E1
y_test_remainder = y_test - E0 - E1_predict
    
sgl = SparseGroupLasso(groups=groups[LAST_POINT_CLUSTER:],
                          lambd=LAMBDA_MIXING)
sgl.set_params(ALPHA)
sgl.fit(X_train[:, LAST_POINT_CLUSTER:], 
            y_train_remainder)
y_train_predict = np.dot(X_train[:, LAST_POINT_CLUSTER:], sgl.coef_)
y_test_predict = np.dot(X_test[:, LAST_POINT_CLUSTER:], sgl.coef_)

print ('TRAINING ERROR:', np.sqrt(mean_squared_error(y_train_predict, y_train_remainder.ravel())))
print ('TESTING ERROR:', np.sqrt(mean_squared_error(y_test_predict, y_test_remainder.ravel())))
coef = np.concatenate(([E0+E1_lasso_CV.intercept_], E1_lasso_CV.coef_, 
                      sgl.coef_), axis=0)
print ('NO. COEFS:', len(np.where(np.abs(coef)>CUTOFF)[0]))

TRAINING ERROR: 0.08116713413292427
TESTING ERROR: 0.06435646896427426
NO. COEFS: 100


In [138]:
np.save('sgl_eci.npy', coef)